In [16]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox

import warnings
warnings.filterwarnings("ignore")

In [17]:
df_stations=pd.read_excel("C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/Data/FullSUPIdentifiedData.xlsx", sheet_name="SUPData", usecols=["stationid","sharedpath", "Latitude",'Longitude'])
#df_stations=df_stations[df_stations.sharedpath==1]

### Combining all

In [71]:
url="C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/Data/Temp/"
road=pd.read_csv(url+"network_density_variables_FULL_condensed_including1516.csv") 
demo1mile=pd.read_csv(url+"FULL_station_and_demographic_1mile.csv") #also includes the google earth collected data
demo05miles=pd.read_csv(url+"FULL_station_and_demographic_05mile.csv") #also includes the google earth collected data
demo025miles=pd.read_csv(url+"FULL_station_and_demographic_025mile.csv") #also includes the google earth collected data
#smrt=pd.read_csv(url+"Full_SmartLocation_data.csv")
route=pd.read_csv(url+"FULL_Transit_route_density.csv")
stop=pd.read_csv(url+"FULL_Transit_stops.csv")
intsct=pd.read_csv(url+"Intersection_count_osm_v2.csv")
lu_replica=pd.read_csv(url+'FULL_replicalanduse_v3.csv')
lu_local=pd.read_csv(url+'FULL_localanduse_labeled_assumed.csv')
lu_local=lu_local.drop(columns={'stationid.1',	'stationid.2'	})
proxi=pd.read_csv(url+'FULL_proximity_data_v4.csv')
job_access=pd.read_csv(url+'FULL_job_access_ped.csv')
job_access_bike=pd.read_csv(url+'FULL_job_access_bike.csv')

In [73]:
job_access.rename(columns={"job_access_15min":"job_access_15min_walk",'job_access_30min':"job_access_30min_walk",'job_access_60min':"job_access_60min_walk"}, inplace=True)
job_access_bike.rename(columns={"job_access_15min":"job_access_15min_bike",'job_access_30min':"job_access_30min_bike",'job_access_60min':"job_access_60min_bike"}, inplace=True)

In [74]:
proxi.rename(columns={'distance_uni':'proximity_uni'}, inplace=True)

In [75]:
demo1mile=demo1mile.drop(['Latitude', 'Longitude'], axis=1) #its already in df_stations
del_cols=['county', 'Latitude', 'Longitude', 'Name', 'Station_Name', 'Same_station_number', 'Direction', 'sharedpath', 'width', 'marking', 'lighting', 'close to freeway', 'benches', 'shading', 'trees', 'one_two_way', 'remark', 'remark_2', 'geometry', 'indexx']
demo05miles=demo05miles.drop(del_cols, axis=1)
demo025miles=demo025miles.drop(del_cols, axis=1)

In [76]:
# merged_file = pd.merge(df_stations, demo, on='stationid')
# print(len(merged_file))
merged_file = pd.merge(lu_local, lu_replica, on='stationid')
print(len(merged_file))
merged_file = pd.merge(merged_file,demo1mile, on='stationid')
print(len(merged_file))
merged_file = pd.merge(merged_file, demo05miles, on='stationid')
print(len(merged_file))
merged_file = pd.merge(merged_file, demo025miles, on='stationid')
print(len(merged_file))

merged_file = pd.merge(merged_file, proxi, on='stationid')
print(len(merged_file))
merged_file = pd.merge(merged_file, job_access, on='stationid', how='outer')
print(len(merged_file))
merged_file = pd.merge(merged_file, job_access_bike, on='stationid', how='outer')
print(len(merged_file))
merged_file = pd.merge(merged_file, intsct, on='stationid', how='outer')
print(len(merged_file))
merged_file = pd.merge(merged_file, route, on='stationid', how='outer')
print(len(merged_file))
merged_file = pd.merge(merged_file, stop, on='stationid', how='outer')
print(len(merged_file))
merged_file = pd.merge(merged_file, road, on='stationid', how='outer')
print(len(merged_file))



665
665
665
665
665
681
681
683
683
683
683


#### Adding count_method (permanent or temporary) and other info for each station

In [77]:
url="C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/Data/BPCX data from Phil/"
stationcode=pd.read_csv(url+"stations as of 202212211541.csv")
stationcode=stationcode[["station_id_tmg", "surface_type_code", "street_lighting_code",'surface_condition_code', "count_method_code"]]
stationtype=stationcode.drop_duplicates(subset='station_id_tmg', keep='first')
stationtype.rename(columns={"station_id_tmg":"stationid",
"surface_type_code":"surface_type", 
"street_lighting_code":"street_lighting",
'surface_condition_code':'surface_condition', 
"count_method_code":"count_method" }, 
inplace=True)


In [78]:
print(len(merged_file.columns))
merged_file = pd.merge(merged_file, stationtype, on='stationid')
print(len(merged_file.columns))

1237
1241


#### Saving 

In [79]:
merged_file.to_csv("Data/Temp/FULL_initial_prepared_data_v13.csv", index=False)
merged_file_SUP=merged_file[merged_file.sharedpath==1]
merged_file_SUP.to_csv("Data/Temp/FULL_initial_prepared_data_SUP_v13.csv", index=False)

### Formating dataset for modeling (The year in the name of columns is extracted and included in the year column)

In [104]:
df=pd.read_csv("Data/Temp/FULL_initial_prepared_data_SUP_v13.csv")
#Here: Format manually between FULL_initial_prepared_data_v7 and FULL_initial_prepared_data_SUP_v7

In [105]:
#ADD NEW VARIABLE HERE
def rename_columns(df):
    new_cols = {col: col.rsplit("_", 1)[0] for col in df.columns}
    df = df.rename(columns=new_cols)
    return df
#The list of columns which which donot have a year at the end of their column names
cols=['stationid','sharedpath', 'Same_station_number', 'count_method',"surface_type", "street_lighting",'surface_condition','width', 'marking', 'lighting', 'close to freeway', 'benches', 'shading', 'trees', 'one_two_way', 'remark', 'remark_2',  
      'Bus_025', 'Tram_strcar_lrail_025', 'Bus_05', 'Tram_strcar_lrail_05', 'Bus_1', 'Tram_strcar_lrail_1',  'stops_025mile', 'stops_05mile', 'stops_1mile'
      ,'Intsct_05mile','Intsct_025mile','Intsct_1mile',
     'proximity_water', 'proximity_uni', 'proximity_school','proximity_parks',
        'job_access_15min_walk', 'job_access_30min_walk', 'job_access_60min_walk','job_access_15min_bike', 'job_access_30min_bike', 'job_access_60min_bike',
        'lu_replica_res_025mile', 'lu_replica_com_025mile',
       'lu_replica_ind_025mile', 'lu_replica_vac_025mile',
       'lu_replica_entropy_025mile',  'lu_replica_res_05mile',
       'lu_replica_com_05mile', 'lu_replica_ind_05mile',
       'lu_replica_vac_05mile', 'lu_replica_entropy_05mile',
       'lu_replica_res_1mile', 'lu_replica_com_1mile', 'lu_replica_ind_1mile',
       'lu_replica_vac_1mile', 'lu_replica_entropy_1mile',
       
       'lu_local_res_025mile', 'lu_local_com_025mile',
       'lu_local_ind_025mile', 'lu_local_vac_025mile',
       'lu_local_entropy_025mile',  'lu_local_res_05mile',
       'lu_local_com_05mile', 'lu_local_ind_05mile',
       'lu_local_vac_05mile', 'lu_local_entropy_05mile', 
       'lu_local_res_1mile', 'lu_local_com_1mile', 'lu_local_ind_1mile',
       'lu_local_vac_1mile', 'lu_local_entropy_1mile',
       
       'lu_local_assumed_025mile','lu_local_assumed_05mile','lu_local_assumed_1mile',
       ]
#2015
df_15 = df.filter(regex='15$')
df_15=rename_columns(df_15)
df_15[cols] = df[cols]
df_15['year'] = 2015

#2016
df_16 = df.filter(regex='16$')
df_16=rename_columns(df_16)
df_16[cols] = df[cols]
df_16['year'] = 2016

#2017
df_17 = df.filter(regex='17$')
df_17=rename_columns(df_17)
df_17[cols] = df[cols]
df_17['year'] = 2017

#2018
df_18 = df.filter(regex='18$')
df_18=rename_columns(df_18)
df_18[cols] = df[cols]
df_18['year'] = 2018

#2019
df_19 = df.filter(regex='19$')
df_19=rename_columns(df_19)
df_19[cols] = df[cols]
df_19['year'] = 2019

#2020
df_20= df.filter(regex='20$')
df_20=rename_columns(df_20)
df_20[cols] = df[cols]
df_20['year'] = 2020

#2021
df_21= df.filter(regex='21$')
df_21=rename_columns(df_21)
df_21[cols] = df[cols]
df_21['year'] = 2021


In [106]:
df_all = pd.concat([df_15,df_16,df_17,df_18,df_19,df_20,df_21])

#### Adding new data (dataset with year column already present)

In [107]:
url="C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/Data/Temp/"
weather=pd.read_csv(url + 'FULL_nearest_weather_data_v4_missingnottreatedbasedonalphabet.csv')

In [108]:
df_all=df_all.merge(weather, on=['stationid', 'year'], how='left')

In [109]:
raw_data=pd.read_csv("Data/Temp/raw_data_.csv")
raw_data=raw_data.rename(columns={"station_id":"stationid"}) 

In [110]:
## Deleting all the stationid in "raw_data" which is not included in df_all and use how="outer" for merging
unwanted_stationids=list(set(raw_data.stationid.unique())-set(df_all.stationid.unique()))
raw_data2 = raw_data[~raw_data['stationid'].isin(unwanted_stationids)]
#Only 2015 to 2021
raw_data3=raw_data2[(raw_data2.year>=2015)&(raw_data2.year<2022)]

#### Adding county information in raw data

In [111]:
url="C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/Data/"
county=pd.read_excel(url + 'FullSUPIdentifiedData.xlsx', usecols=['stationid','county','Longitude', 'Latitude'])

In [112]:
raw_data4=pd.merge(raw_data3, county, on=['stationid'], how='left')
DF2 = pd.merge(raw_data4, df_all, on=['stationid', 'year'], how='left')
DF3=DF2[~DF2.TotPop_1mile.isna()] #This will exclude all data points which doesnot have ACS data (i.e which might be outside of county boundary)

### Treating annual daily traffic data

In [113]:
DF3["ADT_025mile"] = DF3["ADT_CUR_UI_025mile"] + DF3["ADT_CUR_UA_025mile"] + DF3["ADT_CUR_UC_025mile"] + DF3["ADT_CUR_UL_025mile"] +DF3["ADT_CUR_RI_025mile"] + DF3["ADT_CUR_RA_025mile"] + DF3["ADT_CUR_RC_025mile"] + DF3["ADT_CUR_RL_025mile"]
DF3["ADT_05mile"] = DF3["ADT_CUR_UI_05mile"] + DF3["ADT_CUR_UA_05mile"] + DF3["ADT_CUR_UC_05mile"] + DF3["ADT_CUR_UL_05mile"]+DF3["ADT_CUR_RI_05mile"] + DF3["ADT_CUR_RA_05mile"] + DF3["ADT_CUR_RC_05mile"] + DF3["ADT_CUR_RL_05mile"]
DF3["ADT_1mile"] = DF3["ADT_CUR_UI_1mile"] + DF3["ADT_CUR_UA_1mile"] + DF3["ADT_CUR_UC_1mile"] + DF3["ADT_CUR_UL_1mile"]+DF3["ADT_CUR_RI_1mile"] + DF3["ADT_CUR_RA_1mile"] + DF3["ADT_CUR_RC_1mile"] + DF3["ADT_CUR_RL_1mile"]

# Delete the old columns
DF3 = DF3.drop(columns=["ADT_CUR_UI_025mile", "ADT_CUR_UA_025mile", "ADT_CUR_UC_025mile", "ADT_CUR_UL_025mile",
                     "ADT_CUR_UI_05mile", "ADT_CUR_UA_05mile", "ADT_CUR_UC_05mile", "ADT_CUR_UL_05mile",
                     "ADT_CUR_UI_1mile", "ADT_CUR_UA_1mile", "ADT_CUR_UC_1mile", "ADT_CUR_UL_1mile",
                     "ADT_CUR_RI_025mile","ADT_CUR_RI_05mile","ADT_CUR_RI_1mile",
                     "ADT_CUR_RA_025mile","ADT_CUR_RA_05mile","ADT_CUR_RA_1mile",
                     "ADT_CUR_RC_025mile","ADT_CUR_RC_05mile","ADT_CUR_RC_1mile",
                     "ADT_CUR_RL_025mile","ADT_CUR_RL_05mile","ADT_CUR_RL_1mile"
])

#Previous years ADT
DF3["ADT_previous_year_025mile"] = DF3["ADT_previous_year_UI_025mile"] + DF3["ADT_previous_year_UA_025mile"] + DF3["ADT_previous_year_UC_025mile"] + DF3["ADT_previous_year_UL_025mile"] +DF3["ADT_previous_year_RI_025mile"] + DF3["ADT_previous_year_RA_025mile"] + DF3["ADT_previous_year_RC_025mile"] + DF3["ADT_previous_year_RL_025mile"]
DF3["ADT_previous_year_05mile"] = DF3["ADT_previous_year_UI_05mile"] + DF3["ADT_previous_year_UA_05mile"] + DF3["ADT_previous_year_UC_05mile"] + DF3["ADT_previous_year_UL_05mile"]+DF3["ADT_previous_year_RI_05mile"] + DF3["ADT_previous_year_RA_05mile"] + DF3["ADT_previous_year_RC_05mile"] + DF3["ADT_previous_year_RL_05mile"]
DF3["ADT_previous_year_1mile"] = DF3["ADT_previous_year_UI_1mile"] + DF3["ADT_previous_year_UA_1mile"] + DF3["ADT_previous_year_UC_1mile"] + DF3["ADT_previous_year_UL_1mile"]+DF3["ADT_previous_year_RI_1mile"] + DF3["ADT_previous_year_RA_1mile"] + DF3["ADT_previous_year_RC_1mile"] + DF3["ADT_previous_year_RL_1mile"]

#Delete the old columns
DF3 = DF3.drop(columns=["ADT_previous_year_UI_025mile", "ADT_previous_year_UA_025mile", "ADT_previous_year_UC_025mile", "ADT_previous_year_UL_025mile",
"ADT_previous_year_UI_05mile", "ADT_previous_year_UA_05mile", "ADT_previous_year_UC_05mile", "ADT_previous_year_UL_05mile",
"ADT_previous_year_UI_1mile", "ADT_previous_year_UA_1mile", "ADT_previous_year_UC_1mile", "ADT_previous_year_UL_1mile",
"ADT_previous_year_RI_025mile","ADT_previous_year_RI_05mile","ADT_previous_year_RI_1mile",
"ADT_previous_year_RA_025mile","ADT_previous_year_RA_05mile","ADT_previous_year_RA_1mile",
"ADT_previous_year_RC_025mile","ADT_previous_year_RC_05mile","ADT_previous_year_RC_1mile",
"ADT_previous_year_RL_025mile","ADT_previous_year_RL_05mile","ADT_previous_year_RL_1mile",
])

In [114]:
DF3.to_excel("Data/Temp/FULL_prepared_data_SUP_v13.xlsx", index=False)

#### AMpeak AND PMpeak

In [ ]:
#Aggregading direction and calculting ampeak and pmpeak
peakdf=DF3.groupby(['stationid', 'year', 'Mode', 'hour'], as_index=False)['count'].sum()
def peak(x):
    am_peak = x[(x.hour == 10) | (x.hour == 9)]['count'].sum()
    pm_peak = x[(x.hour == 17) | (x.hour == 18)]['count'].sum()
    daily_count= x['count'].sum()
    return pd.Series({'ampeak': am_peak, 'pmpeak': pm_peak, 'daily_count':daily_count})
peakdf2=peakdf.groupby(['stationid', 'year', 'Mode'], as_index=False).apply(peak)
#Calculating daiuly count
data_to_add_with_peakdf=DF3.groupby(['stationid', 'year', 'Mode'], as_index=False).first()
data_to_add_with_peakdf=data_to_add_with_peakdf.drop(['flowid_txdot','hour','count','direction'], axis=1)
#merging both dataset
peakdf3=peakdf2.merge(data_to_add_with_peakdf, on=['stationid','year','Mode'], how='right')

#### Treating missing weather data 

In [ ]:
len(peakdf3)

1290

In [ ]:
#To be treated by fillna(0): all roadway inventory data columns and job access, intersection ..
#To be treated: weather, ADT
missingcols=['surface_type','street_lighting','surface_condition', ]

In [ ]:
# Fill missing values with the mean values of the respective county
for column in ['TAVG', 'TMAX', 'TMIN', 'AWND']:
    peakdf3[column] = peakdf3.groupby('county')[column].transform(lambda x: x.fillna(x.mean()))

# Some values are still missing after location-based treatment, for those, the average of the whole Texas is assigned
mean_values = peakdf3[['TAVG', 'TMAX', 'TMIN', 'AWND']].mean()
peakdf3[['TAVG', 'TMAX', 'TMIN', 'AWND']] = peakdf3[['TAVG', 'TMAX', 'TMIN', 'AWND']].fillna(mean_values)


In [ ]:
peakdf3

,stationid,year,Mode,ampeak,pmpeak,daily_count,county,Latitude,Longitude,TotPop_1mile,...,TAVG,TMAX,TMIN,AWND,ADT_025mile,ADT_05mile,ADT_1mile,ADT_previous_year_025mile,ADT_previous_year_05mile,ADT_previous_year_1mile
0,AL0001,2017,Bic,6.588235,6.359477,38.451007,Collin,33.119987,-96.666206,6385.01131,...,66.3,77.2,55.5,8.9,NaN,NaN,NaN,NaN,NaN,NaN
1,AL0001,2017,Ped,29.660870,18.286957,143.886957,Collin,33.119987,-96.666206,6385.01131,...,66.3,77.2,55.5,8.9,NaN,NaN,NaN,NaN,NaN,NaN
2,AL0001,2018,Bic,5.643836,4.534247,29.701377,Collin,33.119987,-96.666206,6429.97944,...,65.0,75.5,54.6,8.9,NaN,NaN,NaN,NaN,NaN,NaN
3,AL0001,2018,Ped,24.649315,18.232877,134.074101,Collin,33.119987,-96.666206,6429.97944,...,65.0,75.5,54.6,8.9,NaN,NaN,NaN,NaN,NaN,NaN
4,AL0001,2019,Bic,5.505051,4.636364,29.629630,Collin,33.119987,-96.666206,6499.48103,...,64.8,75.4,54.2,8.9,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,WF0001,2018,All,10.636364,5.000000,83.909091,Wichita,33.868206,-98.509804,23571.18323,...,63.8,76.2,51.4,11.2,NaN,NaN,NaN,NaN,NaN,NaN
1286,WF0002,2018,All,4.090909,2.272727,21.454545,Wichita,33.921909,-98.474602,4555.79132,...,63.8,76.2,51.4,11.2,NaN,NaN,NaN,NaN,NaN,NaN
1287,WF0003,2018,Bic,8.714286,5.857143,40.571429,Wichita,33.913994,-98.505325,14570.61666,...,63.8,76.2,51.4,11.2,NaN,NaN,NaN,NaN,NaN,NaN
1288,WF0005,2018,Bic,0.272727,0.181818,1.598485,Wichita,33.958132,-98.603239,2346.31169,...,63.8,76.2,51.4,11.2,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
peakdf3=peakdf3.groupby(['stationid','year','Mode'], as_index=False).agg('first') #There were a lot of dublicates in the data

Adding city

In [ ]:
url="C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/Data/BPCX data from Phil/"
cityinfo=pd.read_csv(url+"_SELECT_s_station_id_tmg_s_flowid_txdot_s_eco_id_s_eco_name_s_se Readable as of 202212211543.csv")
cityinfo=cityinfo[['station_id_tmg','city_area']]

In [ ]:
peakdf4=pd.merge(peakdf3, cityinfo, right_on='station_id_tmg', left_on='stationid', how="left")
peakdf4=peakdf4.drop(["station_id_tmg"], axis=1)
peakdf4=peakdf4.groupby(['stationid','year','Mode'], as_index=False).agg('first')

#### Adding phils data

In [ ]:
#IMPORTS
url='C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/Data/BPCX data from Phil/'
ADT_from_phil=pd.read_csv(url+"ADT_Counts_202304261556.csv")
ADT_from_phil=ADT_from_phil[ADT_from_phil.cmpl1_incmpl0==1] ###################### Filter. The column name suggest complete and incomplete. This might indicate the data collection part.
ADT_from_phil['stationid']=ADT_from_phil.flowid_txdot.str[:6]
ADT_from_phil['Mode']=ADT_from_phil.flowid_txdot.str[-3:]
ADT_from_phil.Mode.replace({"BIc":'Bic'}, inplace=True)

AADT_from_phil=pd.read_csv(url+"AADT_202304261555.csv")
AADT_from_phil=AADT_from_phil[AADT_from_phil.cmpl1_incmpl0==1] ###################### Filter. The column name suggest complete and incomplete. This might indicate the data collection part.
AADT_from_phil['stationid']=AADT_from_phil.flowid_txdot.str[:6]
AADT_from_phil['Mode']=AADT_from_phil.flowid_txdot.str[-3:]

In [ ]:
#Removing dublicate
ADT_from_phil = ADT_from_phil.drop_duplicates(subset=['stationid', 'Mode', 'year'], keep='first')
AADT_from_phil = AADT_from_phil.drop_duplicates(subset=['stationid', 'Mode', 'year'], keep='first')


In [ ]:
#Merge with ADT phil
peakdf5 = pd.merge(peakdf4, ADT_from_phil[['stationid', 'Mode', 'year', 'ADT']], 
                   on=['stationid', 'Mode', 'year'], 
                   how='left')
peakdf5.rename(columns={'ADT': 'ADT_phil'}, inplace=True)# rename the ADT column

# merge with AADT phil
peakdf5 = pd.merge(peakdf5, AADT_from_phil[['stationid', 'Mode', 'year', 'AADT']], 
                   on=['stationid', 'Mode', 'year'], 
                   how='left')
peakdf5.rename(columns={'AADT': 'AADT_phil'}, inplace=True)# rename the AADT column

In [ ]:
peakdf5.to_excel("Data/Temp/FULL_prepared_data_ampm_SUP_v13.xlsx", index=False)

#### Appendix

In [ ]:
#Unique station for each year
table=[]
years=[2015,2016,2017,2018,2019,2020,2021]
for yearz in years:
    num=len(peakdf4[peakdf4.year==yearz].stationid.unique())
    dic={'year':yearz, "unique_statios":num}
    table.append(dic)
Table=pd.DataFrame(table)

##### Check

In [ ]:
condition=(peakdf.stationid=='AU0004') & (peakdf.year==2015) & (peakdf.Mode=='Ped') & (peakdf.hour>8) & (peakdf.hour<11)
peakdf[condition]

,stationid,year,Mode,hour,count
513,AU0004,2015,Ped,9,6.367442
514,AU0004,2015,Ped,10,8.544186


In [ ]:
peakdf[condition]['count'].sum()

14.911627906976744

In [ ]:
condition2=(peakdf2.stationid=='AU0004') & (peakdf2.year==2015) & (peakdf2.Mode=='Ped')
peakdf2[condition2]

,stationid,year,Mode,ampeak,pmpeak,daily_count
21,AU0004,2015,Ped,14.911628,9.046512,115.84186


##### ----